<a href="https://colab.research.google.com/github/Storm00212/Electromagnetics/blob/main/Maxwell's_equations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a simulation of Maxwell's equations using python code to visualize and hence better understand the concepts

In [1]:
# setup of parameters for this simulation
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

# Simulation Parameters
size = 200          # Number of spatial points
steps = 400         # Number of time steps
c0 = 1.0            # Normalized speed of light
dx = 1.0            # Spatial step
dt = dx / (2 * c0)  # Time step (Courant stability condition)

# Initialize Fields
# E and B are shifted by half a grid point (Yee Grid)
E = np.zeros(size)
B = np.zeros(size)


# Faraday's law

In [2]:
def update_B(E, B):
    # B[i] = B[i] - dt/dx * (E[i+1] - E[i])
    B[:-1] -= (dt / dx) * (E[1:] - E[:-1])
    return B
